## Benchmark different algos on different test sets, for structure prediction

One violin plot with the three test sets (PDB, lncRNA, viral_fragments) for each algorithm (RNAstructure, EternaFold, MxFold2, Ufold). 

Group by algorithm.
Colored by test set

**Assigned to**: Alberic

Use Ploty, and a white background

In [11]:
import pandas as pd

results = pd.read_feather('saved_data_plot/results_benchmark_algos.feather').set_index('reference')
# results = results[results['dataset']!= 'lncRNA']
results.loc[results['dataset']=='viral_fragments', 'dataset'] = 'viral mRNA'
results.loc[results['dataset']=='archiveII', 'dataset'] = 'ArchiveII'
# results.loc[results['dataset']=='filtered_PDB', 'dataset'] = 'PDB'
results.loc[results['dataset']=='lncRNA', 'dataset'] = 'long ncRNA'
results.loc[results['model']=='RNAstructure', 'model'] = 'RNAstructure Fold'

models = ['RNAstructure Fold', 'EternaFold', 'MXFold2', 'UFold']

results = results[results['model'].isin(models)]

In [12]:
results.groupby(['model', 'dataset'])[['Precision', 'Recall', 'F1']].mean()

Precision    Recall        F1
model             dataset                                  
EternaFold        ArchiveII    0.573437  0.636239  0.600315
                  PDB          0.883968  0.910543  0.881431
                  long ncRNA   0.449974  0.466777  0.456582
                  viral mRNA   0.750192  0.805463  0.774468
MXFold2           ArchiveII    0.727625  0.760216  0.739838
                  PDB          0.914547  0.928612  0.904817
                  long ncRNA   0.410069  0.429189  0.418591
                  viral mRNA   0.702670  0.720447  0.709519
RNAstructure Fold ArchiveII    0.548525  0.602390  0.571631
                  PDB          0.900336  0.912192  0.891710
                  long ncRNA   0.459832  0.520345  0.487188
                  viral mRNA   0.687321  0.744596  0.712739
UFold             ArchiveII    0.825217  0.886594  0.851515
                  PDB          0.810260  0.965031  0.869775
                  long ncRNA   0.218941  0.143861  0.162995
                  viral mRNA   0.577920  0.587398  0.578628

In [13]:
# Creat a box plot with plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

colors = {'PDB': 'rgb(200, 130, 100)', 
          'ArchiveII': 'rgb(240, 141, 70)',
          'viral mRNA': 'rgb(180, 160, 203)', 
          'long ncRNA': 'rgb(230, 138, 200)', 
          }
for i, dataset in enumerate(results['dataset'].unique()):
    results_dataset = results[results['dataset']==dataset]
    fig.add_trace(go.Violin(x=results_dataset['model'], y=results_dataset['F1'], 
                            name=f'{dataset} (N={len(results_dataset[results_dataset["model"]=="RNAstructure Fold"])})', marker_color=colors[dataset], 
                             meanline_visible=True, points=False))
    
fig.update_layout(
                    # title='F1 score distribution for each model and dataset', 
                  yaxis_title='F1 score', xaxis_title='Model',
                  violinmode='group', yaxis_range=[0, 1],
                  width=1200, height=370,
                  template='plotly_white', font_size=20, font_color='black', font_family='times new roman' )
fig.update_layout(legend_orientation="h", legend=dict(x=0.1, y=1.3))

fig.update_xaxes(categoryorder='array', categoryarray= models)
fig.show()

In [ ]:
# save pdf
fig.write_image("images/b_algo_benchmark.pdf")